In [ ]:
import os
import re
import sys
import glob
import json
import shutil
import skimage
import tifffile
import numpy as np
import pandas as pd
import seaborn as sns

from scipy import ndimage
from matplotlib import pyplot as plt
from matplotlib import colors as mplcolors

from skimage import feature
from skimage import morphology

%load_ext autoreload
%autoreload 2

In [ ]:
tifffile.__version__

In [ ]:
sys.path.append('../')
from pipeline_process.imaging import utils, viz

sys.path.append('/Users/keith.cheveralls/projects/dragonfly-automation/')
import dragonfly_automation.utils
from dragonfly_automation.fov_classifier import FOVClassifier

In [ ]:
# copy only autogained FOVs from 20190910 test to a separate directory
# for creating manual annotations
filepaths = glob.glob(os.path.join(log_dir, 'fov-images', '*.tif'))
print(len(filepaths))

for filepath in filepaths:
    filename = filepath.split(os.sep)[-1]
    new_filepath = os.path.join(out_dir, filename)
    if 'UINT8' in filepath:
        # shutil.copy(filepath, new_filepath)

### 20190910

In [ ]:
# manually annotated good FOVs from 20190910
manual_inds = [
    52,55,
    101,147,175
    ,223,221,224,239,251,262,275,299,
    329,340,375,
    406,415,479,
    503,523,526,539,566,570,580,597,
    650,694,
    705,730,736,775,
    823,841,861,
    920,927,934,960,983,987,999,
    1014,1023,1034,1059,1094,
    1121,1138,1141,1190
]

In [ ]:
log_dir = '/Users/keith.cheveralls/image-data/dragonfly-automation-tests/20190910/ML0000_20190910-3/logs/confluency-check'
out_dir = '/Users/keith.cheveralls/image-data/dragonfly-automation-tests/20190910-snaps-only/'

In [ ]:
# classification log
log = pd.read_csv(os.path.join(log_dir, 'confluency-check-log.csv'))

# fix the filepaths in the classification log
log['filename'] = [os.path.join(log_dir, 'confluency-snaps', name) for name in log.snap_filename]

In [ ]:
log.head()

In [ ]:
# all manual
tile = viz.build_tile(
    log.loc[log.position_ind.isin(manual_inds)], 
    shape=(5, 12), figsize=18)

In [ ]:
# all prediction
tile = viz.build_tile(log.loc[log.confluency_is_good], shape=(6, 12), figsize=18)

In [ ]:
# in manual and prediction
inds = np.array(list(set(prediction_inds).intersection(manual_inds)))
tile = viz.build_tile(log.loc[log.position_ind.isin(inds)], shape=(2, 10), figsize=18)

In [ ]:
# in prediction but not in manual
inds = np.array(list(set(prediction_inds).difference(manual_inds)))
tile = viz.build_tile(log.loc[log.position_ind.isin(inds)], shape=(5, 10), figsize=18)

In [ ]:
# in manual but not in prediction
inds = np.array(list(set(manual_inds).difference(prediction_inds)))
tile = viz.build_tile(log.loc[log.position_ind.isin(inds)], shape=(4, 10), figsize=18)

### 2019-10-03

In [ ]:
log_dir = '/Users/keith.cheveralls/image-data/dragonfly-automation-tests/20191003/ML0000_20191003-3/logs/fov-classification/'
out_dir = '/Users/keith.cheveralls/image-data/dragonfly-automation-tests/20191003-snaps-only/'

In [ ]:
# classification log
log = pd.read_csv(os.path.join(log_dir, 'fov-classification-log.csv'))

# fix the filepaths in the classification log
filepaths = log.image_filepath.values
log['filename'] = [os.path.join(log_dir, 'fov-images', path.split('\\')[-1]) for path in filepaths]

In [ ]:
log.head()

In [ ]:
prediction_inds = log.loc[log.decision].position_ind.values
len(prediction_inds)

In [ ]:
# all prediction
tile = viz.build_tile(log.loc[log.position_ind.isin(prediction_inds)], shape=(6, 12), figsize=18)

### 2019-10-09

This is the first half of plate11 (again). 

In [ ]:
log_dir = '/Users/keith.cheveralls/image-data/dragonfly-automation-tests/ML0000-20191009-2/logs/'
out_dir = '/Users/keith.cheveralls/image-data/dragonfly-automation-tests/ML0000-20191009-2-snaps/'

In [ ]:
# classification log
log = pd.read_csv(os.path.join(log_dir, 'fov-classification', 'fov-classification-log.csv'))
aqlog = pd.read_csv(os.path.join(log_dir, 'ML0000_20191009-2_acquired-images.csv'))

# fix the filepaths in the classification log
filepaths = log.image_filepath.values
log['filename'] = [os.path.join(log_dir, 'fov-classification', 'fov-images', path.split('\\')[-1]) for path in filepaths]

In [ ]:
aqlog.head()

In [ ]:
# exposure times by position_ind
plt.scatter(aqlog.position_ind[1::2], aqlog.exposure_time[1::2])

In [ ]:
# numbers: total FOVs, score > -.5, score > 0, acquired FOVS
log.shape[0], (log.score > -.5).sum(), (log.score > 0).sum(), aqlog.shape[0]/2

In [ ]:
# check that all acquired positions had scores above -0.5 (the threshold we used)
set(aqlog.position_ind).difference(log.loc[log.score > -.5].position_ind)

In [ ]:
# distribution of scores (all and acquired)
bins = np.arange(-1, 1, .2)
_ = plt.hist(log.score, bins=bins, density=True)
_ = plt.hist(log.loc[log.position_ind.isin(aqlog.position_ind)].score, bins=bins, density=True, alpha=.5)

In [ ]:
# number of FOVs per well
d = aqlog.groupby('well_id').count().reset_index()
plt.figure(figsize=(12, 2))
plt.plot(d.well_id, d.timestamp/2)
plt.gca().set_ylim([0, 7])

In [ ]:
d = pd.merge(pinds, log, left_on='ind', right_on='position_ind')
d = d.groupby('well_id').median().reset_index()
plt.figure(figsize=(12, 2))
plt.plot(d.well_id, d.score)
# plt.gca().set_ylim([0, 7])

In [ ]:
# tile acquired FOVs
tile = viz.build_tile(
    log.loc[log.position_ind.isin(aqlog.position_ind)].sort_values(by='score'), 
    shape=(16, 16), 
    figsize=24, 
    offset=0, 
    show_labels=True, 
    label_column='score')

In [ ]:
# tile FOVs with a certain score
tile = viz.build_tile(
    log.loc[log.score < -.5].sort_values(by='score', ascending=False), 
    shape=(20, 16), 
    figsize=24, 
    offset=320*0, 
    show_labels=True, 
    label_column='score')

### 2019-10-23

This is the first half of Plate10. 

In [ ]:
with open('/Users/keith.cheveralls/image-data/dragonfly-automation-tests/ML0000-20191009-2/HCS_sites_20191009_INTERPOLATED.pos') as file:
    pos = json.load(file)

In [ ]:
position_inds = []
for ind, p in enumerate(pos['POSITIONS']):
    well_id, site_num = dragonfly_automation.utils.parse_hcs_site_label(p['LABEL'])
    position_inds.append({'ind': ind, 'well_id': well_id})
    
pinds = pd.DataFrame(data=position_inds)

In [ ]:
log_dir = '/Users/keith.cheveralls/image-data/dragonfly-automation-tests/ML0000-20191023-4//logs/'
out_dir = '/Users/keith.cheveralls/image-data/dragonfly-automation-tests/ML0000-20191023-4-snaps/'

In [ ]:
# classification log
log = pd.read_csv(os.path.join(log_dir, 'fov-classification', 'fov-classification-log.csv'))
aqlog = pd.read_csv(os.path.join(log_dir, 'ML0000_20191023-4_acquired-images.csv'))
afclog = pd.read_csv(os.path.join(log_dir, 'ML0000_20191023-4_afc-calls.csv'))
# fix the filepaths in the classification log
filepaths = log.image_filepath.values
log['filename'] = [os.path.join(log_dir, 'fov-classification', 'fov-images', path.split('\\')[-1]) for path in filepaths]

In [ ]:
afclog.shape, afclog.afc_error.isna().sum()

In [ ]:
plt.scatter(afclog.focusdrive_position_before, afclog.focusdrive_position_after)

In [ ]:
_ = plt.hist(afclog.focusdrive_position_after - afclog.focusdrive_position_before, bins=10)

In [ ]:
_ = plt.hist(afclog.score_before, alpha=.5, bins=np.linspace(-200, 200, 50))
# _ = plt.hist(afclog.score_after, alpha=.5, bins=np.linspace(-200, 200, 50))

In [ ]:
plt.scatter(
    afclog.focusdrive_position_after - afclog.focusdrive_position_before,
    afclog.score_before,
    alpha=.5)

plt.gca().set_ylim([-200, 200])

In [ ]:
afclog.loc[afclog.score_before < -1e3]

In [ ]:
# number of FOVs per well
d = aqlog.groupby('well_id').count().reset_index()
plt.figure(figsize=(12, 2))
plt.plot(d.well_id, d.timestamp/2)
plt.gca().set_ylim([0, 7])

### Mean FOV score by well

In [ ]:
d = pd.merge(pinds, log, left_on='ind', right_on='position_ind')
d = d.groupby('well_id').median().reset_index()
plt.figure(figsize=(12, 2))
plt.plot(d.well_id, d.score)
# plt.gca().set_ylim([0, 7])

In [ ]:
d.sort_values(by='score')

In [ ]:
# tile acquired FOVs
tile = viz.build_tile(
    log.loc[log.position_ind.isin(aqlog.position_ind)].sort_values(by='score'), 
    shape=(12, 16), 
    figsize=24, 
    offset=0, 
    show_labels=True, 
    label_column='score')